In [2]:
import open3d as o3d 
import numpy as np

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [33]:
pcd = o3d.io.read_point_cloud("fragment_000.ply")
print(pcd)
print(np.asarray(pcd.points))
o3d.visualization.draw_geometries([pcd])

PointCloud with 190798 points.
[[ 2.26171875 -0.09253075  1.47265625]
 [ 2.25390625 -0.09085231  1.47265625]
 [ 2.26171875 -0.09179753  1.48046875]
 ...
 [ 0.12109375  0.40638293  1.48828125]
 [ 0.12109375  0.40615749  1.49609375]
 [ 0.12780817  0.38671875  1.49609375]]


In [15]:
print("Original point cloud has", len(pcd.points), "points")


Original point cloud has 190798 points


DownSample

In [19]:
import open3d as o3d

print("Downsample the point cloud with a voxel of 0.001")
downpcd = pcd.voxel_down_sample(voxel_size=0.05)

o3d.visualization.draw_geometries(
    [downpcd],
    zoom=0.8,
    front=[0.4257, -0.2125, -0.8795],
    lookat=[2.6172, 2.0475, 1.532],
    up=[-0.0694, -0.9768, 0.2024]
)


Downsample the point cloud with a voxel of 0.001
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 


Vertex Normal Extimation

In [23]:
print("Recompute the normal of the downsampled point cloud")

downpcd.estimate_normals(
    search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30)
)

o3d.visualization.draw_geometries(
    [downpcd],
    # zoom=0.3412,
    # front=[0.4257, -0.2125, -0.8795],
    # lookat=[2.6172, 2.0475, 1.532],
    # up=[-0.0694, -0.9768, 0.2024],
    point_show_normal=True
)


Recompute the normal of the downsampled point cloud
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 


In [27]:
print("Load a polygon volume and use it to crop the original point cloud")
vol = o3d.visualization.read_selection_polygon_volume("croped.json")
chair = vol.crop_point_cloud(pcd)
o3d.visualization.draw_geometries([chair],
                                zoom=0.7,
                                front=[0.5439, -0.2333, -0.8060],
                                lookat=[2.4615, 2.1331, 1.338],
                                up=[-0.1781, -0.9708, 0.1608])

Load a polygon volume and use it to crop the original point cloud
[Open3D WARNING] The number of points is 0 when creating axis-aligned bounding box.


In [37]:

def create_polygon_volume_programmatically():
    """Create a polygon volume for cropping"""
    print("=== Method 3a: Creating Polygon Volume Programmatically ===")
    
    # Define 2D polygon points (viewed from above, Z-axis)
    polygon_points = np.array([
        [1.5, 1.5],  # Bottom-left
        [3.5, 1.5],  # Bottom-right
        [3.5, 3.0],  # Top-right
        [1.5, 3.0]   # Top-left
    ])
    
    # Create SelectionPolygonVolume
    vol = o3d.visualization.SelectionPolygonVolume()
    vol.bounding_polygon = o3d.utility.Vector2dVector(polygon_points)
    vol.axis_max = 2.0      # Maximum height (Z coordinate)
    vol.axis_min = 0.5      # Minimum height (Z coordinate)
    vol.orthogonal_axis = "Z"  # Extrude along Z-axis
    
    return vol

def crop_with_polygon_volume(pcd):
    """Crop point cloud using polygon volume"""
    print("=== Method 3: Polygon Volume Cropping ===")
    
    # Create polygon volume
    vol = create_polygon_volume_programmatically()
    
    # Crop the point cloud
    cropped_pcd = vol.crop_point_cloud(pcd)
    
    print(f"Points before cropping: {len(pcd.points)}")
    print(f"Points after cropping: {len(cropped_pcd.points)}")
    
    # Save polygon volume to JSON for future use
    o3d.io.write_selection_polygon_volume("my_crop_volume.json", vol)
    print("Saved polygon volume to 'my_crop_volume.json'")
    
    # Visualize
    original_pcd = copy.deepcopy(pcd)
    original_pcd.paint_uniform_color([0.5, 0.5, 0.5])
    cropped_pcd.paint_uniform_color([0.0, 0.0, 1.0])  # Blue
    
    o3d.visualization.draw_geometries([original_pcd, cropped_pcd],
                                    window_name="Polygon Volume Cropping")
    
    return cropped_pcd

In [ ]:
def main():
    """Main function demonstrating all cropping methods"""
    
    # Load or create a sample point cloud
    # Option 1: Load from file
    # pcd = o3d.io.read_point_cloud("your_pointcloud.ply")
    
    # Option 2: Create sample point cloud for demonstration
    print("Creating sample point cloud...")
    mesh = o3d.geometry.TriangleMesh.create_box(width=2.0, height=2.0, depth=2.0)
    mesh.translate([2.0, 2.0, 1.0])
    pcd = mesh.sample_points_uniformly(number_of_points=1000)
    pcd.paint_uniform_color([0.7, 0.7, 0.7])
    
    print(f"Loaded point cloud with {len(pcd.points)} points")
    
    # Show original point cloud
    print("Showing original point cloud...")
    o3d.visualization.draw_geometries([pcd], window_name="Original Point Cloud")
    
    
    
    
    print("All cropping methods demonstrated!")

# ============================================================================
# UTILITY FUNCTIONS
# ============================================================================

def visualize_crop_region(pcd, crop_geometry):
    """Visualize point cloud with crop region"""
    crop_geometry.color = [1, 0, 0]  # Red color for crop region
    o3d.visualization.draw_geometries([pcd, crop_geometry])

def save_cropped_result(cropped_pcd, filename):
    """Save cropped point cloud to file"""
    o3d.io.write_point_cloud(filename, cropped_pcd)
    print(f"Saved cropped point cloud to {filename}")

# Run the example
if __name__ == "__main__":
    main()

Loaded point cloud with 190798 points
Showing original point cloud...
=== Method 3: Polygon Volume Cropping ===
=== Method 3a: Creating Polygon Volume Programmatically ===


TypeError: (): incompatible function arguments. The following argument types are supported:
    1. (self: open3d.cpu.pybind.visualization.SelectionPolygonVolume, arg0: open3d.cpu.pybind.utility.Vector3dVector) -> None

Invoked with: SelectionPolygonVolume, access its members:
orthogonal_axis, bounding_polygon, axis_min, axis_max, std::vector<Eigen::Vector2d> with 4 elements.
Use numpy.asarray() to access data.

Did you forget to `#include <pybind11/stl.h>`? Or <pybind11/complex.h>,
<pybind11/functional.h>, <pybind11/chrono.h>, etc. Some automatic
conversions are optional and require extra headers to be included
when compiling your pybind11 module.